In [4]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (2.3s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
import dotenv

dotenv.load_dotenv()

BINANCE_ETH = "ETHUSDT"
BINANCE_BTC = "BTCUSDT"
BINANCE_XRP = "XRPUSDC"
BINANCE_SOL = "SOLUSDC"
BINANCE_BNB = "BNBUSDC"

BINANCE_SYMBOL = BINANCE_ETH
  
USD_QUOTES = ["SOL", "XRP", "BNB"]
quote_in_usd = len([prefix for prefix in USD_QUOTES if BINANCE_SYMBOL.upper().startswith(prefix)]) > 0

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [27]:
target_period_hours = 24 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 1)

instrument_code = BINANCE_SYMBOL[:-4]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Fri 28 Mar, 08:00 (551 hours left)
current price: 2244.49
-------------------------------
hit ratio: 38%
cost: 0.105 / value: 0.111, gain% = 0.65%
($) cost: 235.67 / value: 250.24, average gain = 14.57
buy put 2100.0
buy call 2300.0
-------------------------------
hit ratio: 35%
cost: 0.073 / value: 0.083, gain% = 0.94%
($) cost: 164.97 / value: 186.12, average gain = 21.15
buy put 2000.0
buy call 2400.0
-------------------------------
hit ratio: 29%
cost: 0.046 / value: 0.057, gain% = 1.08%
($) cost: 103.25 / value: 127.47, average gain = 24.22
buy put 1800.0
buy call 2500.0
-------------------------------
hit ratio: 22%
cost: 0.029 / value: 0.038, gain% = 0.98%
($) cost: 64.19 / value: 86.11, average gain = 21.91
buy put 1500.0
buy call 2600.0


# Data for covered calls

In [28]:
call_data = option_chain_df[["value_call", "value_call_pct", "call_bid"]].copy()
call_data["call_bid_usd"] = option_chain_df["value_call"].mul(option_chain_df["call_bid"].div(option_chain_df["value_call_pct"]))
call_data[["value_call", "call_bid_usd"]]

,value_call,call_bid_usd
strike,,
1500.0,826.772090,NaN
1800.0,542.241775,NaN
2000.0,369.100529,NaN
2100.0,294.264653,NaN
2200.0,230.117537,NaN
2300.0,180.015267,136.913890
2400.0,141.062626,101.002050
2500.0,109.267750,72.945925
2600.0,83.372017,52.745515


# Sanity checks

In [14]:

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
size = 6
option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size=size)
put_weights = [0.] * (2 * size + 1)
call_weights = [0.] * (2 * size + 1)
put_weights[size - 5] = +1.
put_weights[size - 1] = -1.
call_weights[size + 1] = -1.
call_weights[size + 5] = +1.

simulation = trading_model.simulate_strategy(option_chain_df, put_weights, call_weights, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Sun 07 Jul, 08:00 (49 hours left)
current price: 2858.15
-------------------------------
hit ratio: 58%
cost: -0.021 / value: -0.020, gain% = 0.15%
($) cost: -61.45 / value: -57.16, average gain = 4.29
buy put 2600.0
sell put 2800.0
sell call 2900.0
buy call 3000.0


,value_call,call_bid_usd
strike,,
2050.0,157.277626,NaN
2100.0,113.535697,NaN
2150.0,74.588607,NaN
2200.0,44.719136,56.956900
2250.0,26.295392,35.050400
2300.0,15.408865,20.811175
2350.0,9.048906,13.143900
